In [ ]:
import os
import collections
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime

In [ ]:
!pip install -q transformers

In [ ]:
# Importing stock ml libraries
import numpy as np
import pandas as pd
from sklearn import metrics
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel, BertConfig

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

TRAIN D

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
df_train_clinical_P = pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/final_dataset_v4_to_publish/train/trainP.tsv", header=None, sep="\t")

df_train_P = pd.DataFrame()
def load_articles(x,i):
    path = Path(r'/content/drive/MyDrive/Colab Notebooks/final_dataset_v4_to_publish/train/text_files_en') / x[0][i]
    path = str(path) +'.txt'
    df_1 = pd.read_csv(path, header=None, sep="\n")
    df_def = df_1.values[0] 
    for j in range(1, len(df_1)):
        df_def = df_def + df_1.values[j]
    return pd.DataFrame(df_def)

count= 0
for i in range(0, len(df_train_clinical_P)):
    df_2 = load_articles(df_train_clinical_P, i)
    df_train_P = df_train_P.append(df_2)
    count = count + 1


df_train_P = df_train_P.set_axis(np.array(range(0,len(df_train_clinical_P))))
df_train_P[1] = df_train_clinical_P[1]
df_train_P.columns = ['content', 'labels']
df_train_P.head(10)

train = df_train_P

DEV D

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
df_dev_clinical_P = pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/final_dataset_v4_to_publish/dev/devP.tsv", header=None, sep="\t")

df_dev_P = pd.DataFrame()
def load_articles(x,i):
    path = Path(r'/content/drive/MyDrive/Colab Notebooks/final_dataset_v4_to_publish/dev/text_files_en') / x[0][i]
    path = str(path) +'.txt'
    df_1 = pd.read_csv(path, header=None, sep="\n")
    df_def = df_1.values[0] 
    for j in range(1, len(df_1)):
        df_def = df_def + df_1.values[j]
    return pd.DataFrame(df_def)

count= 0
for i in range(0, len(df_dev_clinical_P)):
    df_2 = load_articles(df_dev_clinical_P, i)
    df_dev_P = df_dev_P.append(df_2)
    count = count + 1


df_dev_P = df_dev_P.set_axis(np.array(range(0,len(df_dev_clinical_P))))
df_dev_P[1] = df_dev_clinical_P[1]
df_dev_P.columns = ['content', 'labels']
df_dev_P.head(10)

dev = df_dev_P

TEST D

In [ ]:
from pathlib import Path
df_test_clinical_P = pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/final_dataset_v4_to_publish/test/testP.tsv", header=None, sep="\t")

df_test_P = pd.DataFrame()
def load_articles(x,i):
    path = Path(r'/content/drive/MyDrive/Colab Notebooks/final_dataset_v4_to_publish/test/text_files_en') / x[0][i]
    path = str(path) +'.txt'
    df_1 = pd.read_csv(path, header=None, sep="\n", dtype='unicode', error_bad_lines=False)
    df_def = df_1.values[0] 
    for j in range(1, len(df_1)):
        df_def = df_def + df_1.values[j]
    return pd.DataFrame(df_def)

count= 0
for i in range(0, len(df_test_clinical_P)):
    df_2 = load_articles(df_test_clinical_P, i)
    df_test_P = df_test_P.append(df_2)
    count = count + 1


df_test_P = df_test_P.set_axis(np.array(range(0,len(df_test_clinical_P))))
df_test_P[1] = df_test_clinical_P[1]
LABEL_COLUMNS = ['content', 'labels']
df_test_P.columns = LABEL_COLUMNS
df_test_P.head(10)

test = df_test_P

**MATRIX TRANSFORMATION**

In [ ]:
new_labels_train = []
new_labels_test = []
new_labels_dev = []
lista = ['content']
for i in range(len(df_train_P['labels'])):
    new_labels_train.append([df_train_P['labels'][i]])
    lista = lista + new_labels_train[i]
for i in range(len(df_test_P['labels'])):
    new_labels_test.append([df_test_P['labels'][i]])
    lista = lista + new_labels_test[i]
for i in range(len(df_dev_P['labels'])):
    new_labels_dev.append([df_dev_P['labels'][i]])
    lista = lista + new_labels_dev[i]

    
####### TRAIN #######
myvec = np.zeros((len(df_train_P.drop_duplicates('content', ignore_index = True)),len(lista)))
auxdf = pd.DataFrame(data = myvec, index=range(len(df_train_P.drop_duplicates('content', ignore_index = True))),columns=range(len(lista)))
auxdf.columns = lista
auxdf = auxdf.loc[:,~auxdf.columns.duplicated()]

df_train_P1 = df_train_P
df_train_P = df_train_P.drop_duplicates('content', ignore_index = True)
auxdf['content'] = df_train_P['content']

for j in range(len(auxdf['content'])):
    for i in range(len(df_train_P1['content'])):
        label = df_train_P1['labels'][i]
        if auxdf['content'][j]==df_train_P1['content'][i]:
            label = df_train_P1['labels'][i]
            auxdf[label][j] = 1 
            
train_matrix = auxdf            

#### TEST #####
myvec1 = np.zeros((len(df_test_P.drop_duplicates('content', ignore_index = True)),len(lista)))
auxdf1 = pd.DataFrame(data = myvec1, index=range(len(df_test_P.drop_duplicates('content', ignore_index = True))),columns=range(len(lista)))
auxdf1.columns = lista
auxdf1 = auxdf1.loc[:,~auxdf1.columns.duplicated()]

df_test_P1 = df_test_P
df_test_P = df_test_P.drop_duplicates('content', ignore_index = True)
auxdf1['content'] = df_test_P['content']

for j in range(len(auxdf1['content'])):
    for i in range(len(df_test_P1['content'])):
        label1 = df_test_P1['labels'][i]
        if auxdf1['content'][j]==df_test_P1['content'][i]:
            label1 = df_test_P1['labels'][i]
            auxdf1[label1][j] = 1 
            
test_matrix = auxdf1            

###### DEV ######
myvec3 = np.zeros((len(df_dev_P.drop_duplicates('content', ignore_index = True)),len(lista)))
auxdf3 = pd.DataFrame(data = myvec3, index=range(len(df_dev_P.drop_duplicates('content', ignore_index = True))),columns=range(len(lista)))
auxdf3.columns = lista
auxdf3 = auxdf3.loc[:,~auxdf3.columns.duplicated()]

df_dev_P1 = df_dev_P
df_dev_P = df_dev_P.drop_duplicates('content', ignore_index = True)
auxdf3['content'] = df_dev_P['content']

for j in range(len(auxdf3['content'])):
    for i in range(len(df_dev_P1['content'])):
        label3 = df_dev_P1['labels'][i]
        if auxdf3['content'][j]==df_dev_P1['content'][i]:
            label = df_dev_P1['labels'][i]
            auxdf3[label3][j] = 1 
            
dev_matrix = auxdf3            


##### TRAIN MATRIX #####
df_train_clinical = pd.DataFrame()
# train_matrix = pd.concat([train_matrix, dev_matrix], ignore_index=True)

In [ ]:
train_matrix1 = train_matrix
test_matrix1 = test_matrix
dev_matrix1 = dev_matrix

In [ ]:
total = []
for i in range(train_matrix1.shape[1]):
    if i>0:
        total.append(sum(train_matrix1.iloc[:,i]))


total_matrix = pd.concat([train_matrix1, test_matrix1, dev_matrix1], ignore_index=True)
lista = total_matrix.columns
lista = lista.delete(0)

mat = pd.DataFrame(total, index = lista)
mat = mat.sort_values(0, ascending = False)

maxi = len(lista)
#maxi = 46

coger = mat.iloc[:maxi]
coger = coger.index
coger = coger.insert(0,'content')

train_matrix = train_matrix[coger]
test_matrix = test_matrix[coger]
dev_matrix = dev_matrix[coger]
# print(train_matrix)
# print(test_matrix)

In [ ]:
!pip install texthero
!pip install tweet-preprocessor

In [ ]:
import re 
import nltk
from wordcloud import WordCloud
from nltk.stem import WordNetLemmatizer 
from textblob import TextBlob,Word
from nltk.corpus import words
nltk.download('words')
import texthero as hero
import re
from texthero import stopwords

from nltk.corpus import wordnet

import tensorflow as tf

from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer

import tensorflow as tf


def lemma_per_pos(sent):
    '''function to lemmatize according to part of speech tag'''
    tweet_tokenizer=TweetTokenizer()
    lemmatizer = nltk.stem.WordNetLemmatizer()
    lemmatized_list = [lemmatizer.lemmatize(w) for w in  tweet_tokenizer.tokenize(sent)]
    return " ".join(lemmatized_list)

def df_preprocessing(df,feature_col):
    '''
    Preprocessing of dataframe
    '''
    stop = set(stopwords.words('english'))
    df[feature_col]= (df[feature_col].pipe(hero.lowercase).
                      pipe(hero.remove_urls).
                      pipe(hero.remove_digits).
                      pipe(hero.remove_punctuation).
                      pipe(hero.remove_html_tags) )
    # lemmatization
#     df[feature_col]= [lemma_per_pos(sent) for sent in df[feature_col]]
    # df[col_name]= hero.remove_stopwords(df[col_name],custom_stopwords)
    return df

In [ ]:
target_col= train_matrix.columns[1:]
feature_col= train_matrix.columns[0]

In [ ]:
with tf.device('/GPU:0'):
    proc_train_df= df_preprocessing(train_matrix,feature_col)

In [ ]:
proc_test_df = df_preprocessing(test_matrix,feature_col)

In [ ]:
from transformers import AutoTokenizer,TFDistilBertModel, DistilBertConfig
from transformers import TFAutoModel
import tensorflow as tf 
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from transformers import AdamW, get_linear_schedule_with_warmup
#import tensorflow_addons as tfa


#Creating tokenizer
def create_tokenizer(pretrained_weights='distilbert-base-uncased'):
  '''Function to create the tokenizer'''

  tokenizer = AutoTokenizer.from_pretrained(pretrained_weights)
  return tokenizer

#Tokenization of the data
def data_tokenization(dataset,feature_col,max_len,tokenizer):
    '''dataset: Pandas dataframe with feature name is column name 
    Pretrained_weights: selected model 
    RETURN: [input_ids, attention_mask]'''

    tokens = dataset[feature_col].apply(lambda x: tokenizer(x,return_tensors='tf', 
                                                            truncation=True,
                                                            padding='max_length',
                                                            max_length=max_len, 
                                                            add_special_tokens=True))
    input_ids= []
    attention_mask=[]
    for item in tokens:
        input_ids.append(item['input_ids'])
        attention_mask.append(item['attention_mask'])
    input_ids, attention_mask=np.squeeze(input_ids), np.squeeze(attention_mask)


    return [input_ids,attention_mask]

def bert_model(pretrained_weights,max_len,learning_rate):
  '''BERT model creation with pretrained weights
  INPUT:
  pretrained_weights: Language model pretrained weights
  max_len: input length '''
  print('Model selected:', pretrained_weights)
  bert=TFAutoModel.from_pretrained(pretrained_weights)
  
  # This is must if you would like to train the layers of language models too.
  for layer in bert.layers:
      layer.trainable = True

  # parameter declaration
#   step = tf.Variable(0, trainable=False)
#   schedule = tf.optimizers.schedules.PiecewiseConstantDecay([10000, 15000], [2e-0, 2e-1, 1e-2])
#   # lr and wd can be a function or a tensor
#   lr = learning_rate * schedule(step)
#   wd = lambda:lr * schedule(step)
#   optimizer = tfa.optimizers.AdamW(learning_rate=lr, weight_decay=wd)

  optimizer= tf.keras.optimizers.Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,name='Adam')
#   optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate)

  # declaring inputs, BERT take input_ids and attention_mask as input
  input_ids= Input(shape=(max_len,),dtype=tf.int32,name='input_ids')
  attention_mask=Input(shape=(max_len,),dtype=tf.int32,name='attention_mask')

  bert= bert(input_ids,attention_mask=attention_mask)
  x= bert[0][:,0,:]
  x=tf.keras.layers.Dropout(0.1)(x)
  x= tf.keras.layers.Dense(128)(x)
  x=tf.keras.layers.Dense(64)(x)
  x=tf.keras.layers.Dense(32)(x)

  output=tf.keras.layers.Dense(maxi,activation='sigmoid')(x)

  model=Model(inputs=[input_ids,attention_mask],outputs=[output])
  # compiling model 
  model.compile(optimizer=optimizer,
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True, reduction=tf.keras.losses.Reduction.NONE,name='binary_crossentropy'),
                metrics=['accuracy'])
  return model


In [ ]:
pretrained_weights='bert-base-uncased'
max_len=256
epochs=20
learning_rate=2e-5
batch_size=4

In [ ]:
tokenizer= create_tokenizer(pretrained_weights)

In [ ]:
x_train= data_tokenization(proc_train_df,feature_col,max_len,tokenizer)

In [ ]:
y_train= proc_train_df[target_col].values
y_train

In [ ]:
bert=bert_model(pretrained_weights,max_len,learning_rate)
bert.summary()

In [ ]:
with tf.device('/GPU:0'):
    bert.fit(x_train,y_train,batch_size=batch_size,epochs=epochs,verbose=1)

In [ ]:
x_test= data_tokenization(test_matrix,feature_col,max_len,tokenizer)
x_test

In [ ]:
preds= bert.predict(x_test)
submiss_df= pd.DataFrame(preds, columns= target_col)
submiss_df

In [ ]:
outputs = submiss_df>0.5
f1_score_micro = metrics.f1_score(test_matrix.iloc[:,1:], outputs, average='micro')
f1_score_macro = metrics.f1_score(test_matrix.iloc[:,1:], outputs, average='macro')
ap = metrics.average_precision_score(test_matrix.iloc[:,1:], outputs, average='micro')
print(f1_score_micro)
print(f1_score_macro)
print(ap)